### Initilization

In [1]:
import pypsa
import pandas as pd
import glob

### Scenarios

In [2]:
# --------- 1) Scenarios  ----------
# base:   grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         nothing extendable
# 2:      grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         elec flow storage 0.72MWh, 0.12MW
#         nothing extendable
# 3:      grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         elec flow storage 0.72MWh, 0.12MW
#         rooftop PV extendable
# 4:      No grid connection
#         CHP extendable
#         solar_freefield 0.840 MWp
#         elec flow storage extendable
#         rooftop PV extendable
# 5:      No grid connection
#         CHP extendable
#         solar_freefield 0.840 MWp
#         elec flow storage extendable
#         rooftop PV extendable
#         wind extendable

pars = {
    1: dict(grid=True,  chp_ext=False, solar_roof=False, flow_storage=False, flow_storage_ext=False, wind=False),
    2: dict(grid=True,  chp_ext=False, solar_roof=False, flow_storage=True,  flow_storage_ext=False, wind=False),
    3: dict(grid=True,  chp_ext=False, solar_roof=True,  flow_storage=True,  flow_storage_ext=False, wind=False),
    4: dict(grid=False, chp_ext=True,  solar_roof=True,  flow_storage=True,  flow_storage_ext=True,  wind=False),
    5: dict(grid=False, chp_ext=True,  solar_roof=True,  flow_storage=True,  flow_storage_ext=True,  wind=True)
}

wd = "/home/cs/Documents/Cola/Master/MS_5-Semester/AU-Viborg" # working directory

### Data

In [21]:
# Interest rate
ir = 0.07

# Raw cost data
data = pd.DataFrame({
    "capex": [400e3, 1000e3, 1100e3, 400e3, 500e3, 8, 0],  # eur/MWp eur/L
    "FOM": [10e3, 12e3, 14e3, 14e3, 10e3, 0, 0],  # Fixed O&M (€/MWp/year)
    "opex": [0, 0, 0.05e3, 0.17e3, 0, 0, 0.35e3],  # Variable O&M (€/MWh)
    "lifetime": [35, 35, 27, 20, 20, 20, 1],  # Grid lifetime=1 to avoid div/0
}, index=["PVFree", "PVRoof", "onwind", "CHP", "flow_storage", "thermal_storage", "grid"])

# Annualize CAPEX and add FOM
# For technologies with capital investment: annualized_capex = CAPEX * CRF + FOM
# CRF (Capital Recovery Factor) = ir*(1+ir)^n / ((1+ir)^n - 1)
data["capex_annual"] = data["capex"] * (ir * (1 + ir)**data["lifetime"] / 
                                         ((1 + ir)**data["lifetime"] - 1)) + data["FOM"]

# Grid costs need special handling
# Since PyPSA optimizes per year, we treat the 40 €/kW as an annual capacity cost
grid_capacity_cost = 40e3  # €/MW annual capacity charge
grid_energy_cost = 0.35e3  # €/MWh energy charge

# Wind timeseries
wind_ts = pd.read_csv(f"{wd}/data/Wind_Vestas-V90-2000_2019.csv", comment="#")
wind_ts = wind_ts.set_index("time")
wind_ts.index = pd.to_datetime(wind_ts.index)
wind_ts = wind_ts.drop(columns=["local_time"])
wind_ts["CF"] = wind_ts["electricity"]/2000 # 2000 rated power

# Solar timeseries
solar_dict = {'SE': 2.003, 'S': 1.056, 'SW': 6.643, 'E': 0.731, 'NW': 2.003, 'N': 1.056, 'NE': 6.643, 'W': 0.731} #MW
solar_ts_dict = {}
for loc in solar_dict:  # loop directly over dictionary keys
    files = glob.glob(f"{wd}/data/Solar_{loc}*.csv")
    if not files:
        continue
    ts = pd.read_csv(files[0], skiprows=10, skipfooter=11, engine="python")
    ts["time"] = pd.to_datetime(ts["time"].str.replace(":", ""), format="%Y%m%d%H%M")
    ts["time"] = ts["time"].dt.floor("h")
    ts = ts.set_index("time")
    # Compute capacity factor using the dictionary
    solar_ts_dict[loc] = ts["P"] / (solar_dict[loc] * 1000)
solar_ts = pd.DataFrame(solar_ts_dict) # Combine all location series into one DataFrame

# Heat demand timeseries
L_h = pd.read_excel(f"{wd}/data/Synthese_von_Waermelastprofilen_nach_BDEW.xlsx", sheet_name="Ausgabe")
#L_h["Energie in kWh"].sum() # check if it is 626.57 m3 * 10.16 kWh/m3 = 6365.95 kWh as given in the data
L_h = L_h.rename(columns={"Zeitstempel": "time", "Energie in kWh": "energy"})
L_h = L_h.set_index('time')
L_h.index = pd.to_datetime(L_h.index, unit='h')
L_h.index = L_h.index - pd.DateOffset(years=4) #match 2019 timestamp
L_h = L_h.iloc[:-24] # drop the day after newyears
L_h = L_h/1000 # convert to MWh

# Power Demand Timeseries
# Based on that there as of now is no CHP (so full elec demand as given by Björn)
L_e = pd.read_excel(f"{wd}/data/N61616_sum_alle_3.xlsx")
L_e = L_e.iloc[:, [4,8]]
L_e = L_e.rename(columns={"Gældende fra": "time", "Sum 61616 ": "energy"}) 
L_e['time'] = pd.to_datetime(L_e['time']).dt.tz_localize("Europe/Copenhagen", ambiguous="NaT", nonexistent="NaT")
L_e['time'] = L_e['time'].dt.tz_convert("UTC")
L_e.index = L_e["time"]
L_e = L_e.drop('time', axis=1)
last_row = L_e.iloc[[-1]].copy()
last_row.index = last_row.index + pd.Timedelta(hours=1)
L_e = pd.concat([L_e, last_row])
L_e.index = wind_ts.index
L_e = L_e/1000 # convert kWh to MWh

### Network setup

In [24]:
# Choose Scenario
SC = 1 # 1,2,3,4,5
cfg = pars[SC]

In [ ]:
# --------- Setup Network ----------
n = pypsa.Network()
n.set_snapshots(wind_ts.index)

# Buses
n.add("Bus", "elec")
n.add("Bus", "heat")
n.add("Bus", "gas")

# Loads (use your actual load data)
n.add("Load", "L_e", bus="elec", p_set=L_e["energy"])
n.add("Load", "L_h", bus="heat", p_set=L_h["energy"])

# --------- Generators ----------
# Grid import
if cfg.get("grid") == True:
      n.add("Carrier", "grid")
      n.add("Generator",
            "grid_import",
            bus="elec",
            carrier="grid",
            p_nom_extendable=True,
            p_nom_max=1e9,
            marginal_cost=grid_energy_cost,  # 0.35e3 eur/MWh
            capital_cost=grid_capacity_cost)  # 40e3 eur/MW annual

# Solar freefield
n.add("Carrier", "solar")
n.add("Generator",
      "solar_S_Free",
      bus="elec",
      carrier="solar",
      p_max_pu=solar_ts["S"],
      capital_cost=data.at["PVFree", "capex_annual"],
      p_nom_extendable=True,
      p_nom_max=0.84) #MWp

# Solar generators (rooftop)
if cfg.get("solar_roof") == True:
      for loc in solar_ts.columns:
            n.add("Generator",
                  f"solar_{loc}",
                  bus="elec",
                  carrier="solar",
                  p_max_pu=solar_ts[loc],
                  capital_cost=data.at["PVRoof", "capex_annual"],
                  p_nom_extendable=True,
                  p_nom_max=solar_dict[loc])

# Wind
if cfg.get("wind") == True:
      n.add("Carrier", "wind")
      n.add("Generator",
            "wind",
            bus="elec",
            carrier="wind",
            p_max_pu=wind_ts["CF"],
            capital_cost=data.at["onwind", "capex_annual"],
            p_nom_extendable=True,)
            #p_nom_max=2) # own estimate

# Biogas supply
n.add("Carrier", "biogas")
n.add("Generator",
      "biogas_supply",
      bus="gas",
      carrier="biogas",
      p_nom=1e9,
      marginal_cost=0)  # Cost is in CHP operation

# --------- Links ----------
# CHP: gas -> electricity (40%) + heat (50%)
n.add("Carrier", "CHP")
n.add("Link",
      "CHP",
      carrier="CHP",
      bus0="gas",
      bus1="elec",
      bus2="heat",
      p_nom=1.5875,  # 0.635 MW_el / 0.4
      efficiency=0.40,
      efficiency2=0.50,
      capital_cost=data.at["CHP", "capex_annual"],  # eur/MW_th/year
      marginal_cost=data.at["CHP", "opex"],  # 0.17e3 eur/MWh biogas cost
      p_nom_extendable=cfg.get("chp_ext"))

# heatsink (dump heat)
n.add("Carrier", "heat_sink")
n.add("StorageUnit",
    "heat_sink",
    bus="heat",
    carrier="heat_sink",
    max_hours=1e9,
    capital_cost=0,
    efficiency_store=1,
    efficiency_dispatch=0.01,
    standing_loss=1,
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
)

# --------- Storage ----------
# Flow storage (battery) - using Store + Links
# Store: holds energy (€/MWh cost)
# Links: charge and discharge (€/kW cost)
if cfg.get("flow_storage") == True:
      n.add("Bus", "battery")

      # Add the Store (energy capacity)
      n.add("Store",
            "flow_storage",
            bus="battery",
            e_nom=0.72,  # Energy capacity in MWh
            e_cyclic=True,  # State of charge is cyclic (end = start)
            capital_cost= data.at["flow_storage", "capex_annual"],  # €/MWh/year
            e_nom_extendable=cfg.get("flow_storage_ext"))

      # Add charging Link (grid -> battery)
      n.add("Link",
            "flow_storage_charge",
            bus0="elec",
            bus1="battery",
            p_nom=0.12,  # Charging power in MW
            efficiency=0.9,  # Charging efficiency
            capital_cost=0)  # €/MW/year for charging
            #p_nom_extendable=cfg.get("flow_storage_ext"))

      # Add discharging Link (battery -> grid)
      n.add("Link",
            "flow_storage_discharge",
            bus0="battery",
            bus1="elec",
            p_nom=0.12,  # Discharging power in MW
            efficiency=0.9,  # Discharging efficiency
            capital_cost=0)  # €/MW/year for discharging
            #p_nom_extendable=cfg.get("flow_storage_ext"))

# Print cost summary
print("\n=== Annualized Costs (€/MW/year or €/MWh) ===")
print(data[["capex", "FOM", "capex_annual", "opex"]])
print(f"\nGrid: {grid_capacity_cost} €/MW + {grid_energy_cost} €/MWh")
# print(f"\nFlow storage (Store+Links):")
# print(f"  - Energy: {flow_energy_capex} €/MWh/year for {flow_energy_capacity} MWh")
# print(f"  - Power: {flow_power_capex} €/MW/year for {flow_power_capacity} MW charge/discharge")


=== Annualized Costs (€/MW/year or €/MWh) ===
                     capex      FOM   capex_annual   opex
PVFree            400000.0  10000.0   40893.583860    0.0
PVRoof           1000000.0  12000.0   89233.959649    0.0
onwind           1100000.0  14000.0  105768.307420   50.0
CHP               400000.0  14000.0   51757.170297  170.0
flow_storage      500000.0  10000.0   57196.462872    0.0
thermal_storage        8.0      0.0       0.755143    0.0
grid                   0.0      0.0       0.000000  350.0

Grid: 40000.0 €/MW + 350.0 €/MWh


In [29]:
n.loads_t.p_set

name,L_e,L_h
snapshot,,
2019-01-01 00:00:00,0.160829,0.000786
2019-01-01 01:00:00,0.157510,0.001008
2019-01-01 02:00:00,0.152188,0.001008
2019-01-01 03:00:00,0.150835,0.001151
2019-01-01 04:00:00,0.154732,0.001452
...,...,...
2019-12-31 19:00:00,0.131966,0.000991
2019-12-31 20:00:00,0.128252,0.000975
2019-12-31 21:00:00,0.125656,0.000916


In [30]:
n.optimize(solver_name='gurobi', keep_files=False)

Index(['elec', 'heat', 'gas'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 175.08it/s]
INFO:linopy.io: Writing time: 0.74s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


INFO:gurobipy:Set parameter WLSSecret


...

INFO:gurobipy:Solved in 1009 iterations and 1.72 seconds (0.53 work units)


Optimal objective  2.591150457e+05


INFO:gurobipy:Optimal objective  2.591150457e+05


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 61323 primals, 157685 duals
Objective: 2.59e+05
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

In [ ]:
generators_capital_costs = (n.generators['capital_cost'] * (n.generators['p_nom_opt'] - n.generators['p_nom'])).groupby(n.generators['carrier']).sum()
generators_capital_costs
n.links.capital_cost
generators_marginal_costs = (n.generators['marginal_cost'] * n.generators_t.p.sum(axis=0)).groupby(n.generators['carrier']).sum()
generators_marginal_costs

carrier
biogas         0.000000
grid      245549.516646
solar          0.000000
dtype: float64

In [ ]:
n.generators_t.

{'p_min_pu': Empty DataFrame
 Columns: []
 Index: [2019-01-01 00:00:00, 2019-01-01 01:00:00, 2019-01-01 02:00:00, 2019-01-01 03:00:00, 2019-01-01 04:00:00, 2019-01-01 05:00:00, 2019-01-01 06:00:00, 2019-01-01 07:00:00, 2019-01-01 08:00:00, 2019-01-01 09:00:00, 2019-01-01 10:00:00, 2019-01-01 11:00:00, 2019-01-01 12:00:00, 2019-01-01 13:00:00, 2019-01-01 14:00:00, 2019-01-01 15:00:00, 2019-01-01 16:00:00, 2019-01-01 17:00:00, 2019-01-01 18:00:00, 2019-01-01 19:00:00, 2019-01-01 20:00:00, 2019-01-01 21:00:00, 2019-01-01 22:00:00, 2019-01-01 23:00:00, 2019-01-02 00:00:00, 2019-01-02 01:00:00, 2019-01-02 02:00:00, 2019-01-02 03:00:00, 2019-01-02 04:00:00, 2019-01-02 05:00:00, 2019-01-02 06:00:00, 2019-01-02 07:00:00, 2019-01-02 08:00:00, 2019-01-02 09:00:00, 2019-01-02 10:00:00, 2019-01-02 11:00:00, 2019-01-02 12:00:00, 2019-01-02 13:00:00, 2019-01-02 14:00:00, 2019-01-02 15:00:00, 2019-01-02 16:00:00, 2019-01-02 17:00:00, 2019-01-02 18:00:00, 2019-01-02 19:00:00, 2019-01-02 20:00:00, 2019